In [12]:
import spacy
from spacy.lang.es import Spanish
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Span, Token, Doc
from spacy.language import Language
from spacy.training import Example
import json
import random

## Capítulo 1: Encontrando palabras, frases, nombres y conceptos

In [6]:
nlp = Spanish()

doc = nlp("Quiero una dieta baja en sodio!")

print(doc.text)

Quiero una dieta baja en sodio!


### Atributos léxicos

In [9]:
words = [token.is_alpha for token in doc]
symbols = [token.is_punct for token in doc]
print(words)
print(symbols)

[True, True, True, True, True, True, False]
[False, False, False, False, False, False, True]


### MODELOS

In [10]:
nlp = spacy.load("es_core_news_sm")
text = (
    "De acuerdo con la revista Fortune, Apple fue la empresa "
    "más admirada en el mundo entre 2008 y 2012."
)

# Procesa el texto
doc = nlp(text)

# Imprime en pantalla el texto del documento
print(doc.text)

for token in doc:
    # Obtén el texto del token, el part-of-speech tag y el dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # Esto es solo por formato
    print("{:<12}{:<10}{:<10}".format(token_text, token_pos, token_dep))

# Itera sobre las entidades predichas
for ent in doc.ents:
    # Imprime en pantalla el texto de la entidad y su label
    print(ent.text, ent.label_)

years = doc[-4:]
print("Entidad faltante:", years.text)

De acuerdo con la revista Fortune, Apple fue la empresa más admirada en el mundo entre 2008 y 2012.
De          ADP       case      
acuerdo     NOUN      fixed     
con         ADP       fixed     
la          DET       det       
revista     NOUN      obl       
Fortune     PROPN     appos     
,           PUNCT     punct     
Apple       PROPN     nsubj     
fue         AUX       cop       
la          DET       det       
empresa     NOUN      ROOT      
más         ADV       advmod    
admirada    ADJ       amod      
en          ADP       case      
el          DET       det       
mundo       NOUN      obl       
entre       ADP       case      
2008        NOUN      obl       
y           CCONJ     cc        
2012        NOUN      conj      
.           PUNCT     punct     
revista Fortune ORG
Apple ORG
Entidad faltante: 2008 y 2012.


### PATRONES

In [53]:
matcher = Matcher(nlp.vocab)
pattern = [{"LIKE_NUM": True}]
matcher.add("PATTERN", [pattern])
matches = matcher(doc)
print(f'Resultados: {[doc[start:end].text for match_id, start, end in matches]}')

pattern = [{"POS": "NOUN"}, {"POS": "PROPN"}]
matcher.add("NOUN_ADJ_PATTERN", [pattern])
matches = matcher(doc)
print(f'Resultados: {[doc[start:end].text for match_id, start, end in matches]}')

Resultados: ['2008', '2012']
Resultados: ['revista Fortune', '2008', '2012']


## Capítulo 2: Análisis de datos a gran escala con spaCy

In [54]:
revista_hash = doc.vocab.strings["revista"]
print(revista_hash)
print(doc.vocab.strings[revista_hash])

1212018860762523042
revista


In [60]:
# Itera sobre los tokens
for token in doc:
    if token.pos_ == "NOUN":
        if doc[token.i -1].pos_ == "DET":
            print("Encontré un nombre propio antes de un verbo:", token.text)

Encontré un nombre propio antes de un verbo: revista
Encontré un nombre propio antes de un verbo: empresa
Encontré un nombre propio antes de un verbo: mundo


In [76]:
# Carga el modelo es_core_news_md
nlp = spacy.load("es_core_news_md")

# Procesa un texto
doc = nlp("Hoy hice pan de cereal")

# Obtén el vector para el token "cereal"
# cereal_vector = doc[4].vector
# print(cereal_vector)

doc = nlp("coca-cola y refresco")
token1, token2 = doc[0], doc[2]

# Obtén la similitud entre los tokens "TV" y "libros"
similarity = token1.similarity(token2)
print(similarity)

0.6222537


In [5]:
nlp = spacy.load("es_core_news_sm")
doc = nlp(
    "Cuando pac-man debutó en Tokio, en 1980, nadie podría haber predicho "
    "que se convertiría en el videojuego más exitoso de todos los tiempos. "
    "Hoy, 40 años después, aun sigue sorprendiendo. Su desarrolladora, "
    "Bandai Namco, ha anunciado novedades en el marco del aniversario del "
    "juego. La celebración del 40 aniversario de pac-Man en 2020 incluirá "
    "el debut de una nueva canción temática, compuesta por el famoso artista "
    "japonés de Techno Ken Ishii. Además de estas novedades, Bandai Namco "
    "publicará nuevas versiones del videojuego. La primera será pac-man Live "
    "Studio, en Twitch, en colaboración con Amazon Games."
)

# Crea los patrones
pattern1 = [{"LIKE_NUM": True}, {"POS": "NOUN"}]
pattern2 = [{"LOWER": "pac-man"}, {"IS_TITLE": True}]

# Inicializa el Matcher y añade los patrones
matcher = Matcher(nlp.vocab)
matcher.add("PATTERN1", [pattern1])
matcher.add("PATTERN2", [pattern2])

# Itera sobre los resultados
for match_id, start, end in matcher(doc):
    # Imprime en pantalla el nombre en string del patrón
    # y el texto del span encontrado
    print(doc.vocab.strings[match_id], doc[start:end].text)

PATTERN1 40 años
PATTERN1 40 aniversario
PATTERN2 pac-man Live


In [8]:
with open("../resources/notebook/countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())

nlp = Spanish()
doc = nlp(
    "La Unión Europea fue fundada por seis países de Europa occidental "
    "(Francia, Alemania, Italia, Bélgica, Países Bajos, y Luxemburgo) y "
    "se amplió en seis ocasiones."
)

matcher = PhraseMatcher(nlp.vocab)
# Crea objetos Doc patrón y añádelos al matcher
# Esta es una versión más rápida de: [nlp(country) for country in COUNTRIES]
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", None, *patterns)
# Llama al matcher sobre el documento de prueba e imprime el resultado en pantalla
matches = matcher(doc)
print([doc[start:end] for match_id, start, end in matches])

[Francia, Alemania, Italia, Bélgica, Países Bajos, Luxemburgo]


In [14]:
with open("../resources/notebook/country_text.txt", encoding="utf8") as f:
    TEXT = f.read()

nlp = spacy.load("es_core_news_sm")
matcher = PhraseMatcher(nlp.vocab)
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", None, *patterns)

# Crea un doc y restablece las entidades existentes
doc = nlp(TEXT)
doc.ents = []

# Itera sobre los resultados
for match_id, start, end in matcher(doc):
    # Crea un Span con el label para "LOC"
    span = Span(doc, start, end, label="LOC")

    # Sobrescribe el doc.ents y añade el span
    doc.ents = list(doc.ents) + [span]

    # Obtén el token cabeza de la raíz del span
    span_root_head = span.root.head

    # Imprime en pantalla el texto del token cabeza de
    # la raíz del span y el texto del span
    print(span_root_head.text, "-->", span.text)

# Imprime en pantalla las entidades del documento
print([(ent.text, ent.label_) for ent in doc.ents if ent.label_ == "LOC"])

Brasil --> Brasil
Brasil --> Alemania
Brasil --> India
Brasil --> Japón
segundos --> Japón
Japón --> Alemania
dos --> Brasil
Brasil --> India
incluir --> Brasil
Brasil --> Alemania
Brasil --> India
Brasil --> Japón
habitualmente --> Egipto
Egipto --> Nigeria
apoyado --> Estados Unidos
membresía --> Japón
apoyo --> India
apoyan --> Reino Unido
Reino --> Francia
acceso --> Alemania
Alemania --> Brasil
Alemania --> India
Alemania --> Japón
apoyado --> China
membresía --> Japón
puja --> India
apoyo --> Francia
Francia --> Rusia
Francia --> Reino Unido
Francia --> Estados Unidos
adquirido --> India
expresó --> China
expresado --> China
candidatura --> India
revoca --> India
[('Brasil', 'LOC'), ('Alemania', 'LOC'), ('India', 'LOC'), ('Japón', 'LOC'), ('Japón', 'LOC'), ('Alemania', 'LOC'), ('Brasil', 'LOC'), ('India', 'LOC'), ('Brasil', 'LOC'), ('Alemania', 'LOC'), ('India', 'LOC'), ('Japón', 'LOC'), ('Egipto', 'LOC'), ('Nigeria', 'LOC'), ('Estados Unidos', 'LOC'), ('Japón', 'LOC'), ('India',

## Capítulo 3: Pipelines de procesamiento

In [2]:
nlp = spacy.load("es_core_news_sm")

print(nlp.pipe_names)
print(nlp.pipeline)

['tok2vec', 'morphologizer', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']
[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x0000025D99A8C8B0>), ('morphologizer', <spacy.pipeline.morphologizer.Morphologizer object at 0x0000025D99ACC5E0>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x0000025D99A64B80>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x0000025D999B2D60>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x0000025D99A74BC0>), ('lemmatizer', <spacy.lang.es.lemmatizer.SpanishLemmatizer object at 0x0000025D99A6D1C0>)]


### Componentes personalizados para el pipeline

In [7]:
@Language.component("length_component")
def length_component(doc):
    doc_length = len(doc)
    print(f"Este documento tiene {doc_length} tokens.")
    return doc


# Añade el componente en el primer lugar del pipeline e imprime
# los nombres de los pipes en pantalla
nlp.add_pipe("length_component", name="print_length", first=True)
print(nlp.pipe_names)

# Procesa un texto
doc = nlp("Esto es una frase.")

['print_length', 'tok2vec', 'morphologizer', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']
Este documento tiene 5 tokens.


In [9]:

# Define el componente personalizado
@Language.component("animal_component")
def animal_component(doc):
    # Aplica el matcher al doc
    matches = matcher(doc)
    # Crea un Span para cada resultado y asígna el label "ANIMAL"
    spans = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    # Sobrescribe los doc.ents con los spans resultantes
    doc.ents = spans
    return doc

nlp = spacy.load("es_core_news_sm")

animals = ["labrador dorado", "gato", "tortuga", "oso de anteojos"]
animal_patterns = list(nlp.pipe(animals))
print("animal_patterns:", animal_patterns)
matcher = PhraseMatcher(nlp.vocab)
matcher.add("ANIMAL", None, *animal_patterns)

# Añade el componente al pipeline después del componente "ner"
nlp.add_pipe("animal_component", name="animal_matcher", after="ner")
print(nlp.pipe_names)

# Procesa el texto e imprime en pantalla el texto y el label
# de los doc.ents
doc = nlp("Hoy vimos una tortuga y un oso de anteojos en nuestra caminata")
print([(ent.text, ent.label_) for ent in doc.ents])

animal_patterns: [labrador dorado, gato, tortuga, oso de anteojos]
['tok2vec', 'morphologizer', 'parser', 'ner', 'animal_matcher', 'attribute_ruler', 'lemmatizer']
[('tortuga', 'ANIMAL'), ('oso de anteojos', 'ANIMAL')]


### Atributos personalizados

In [12]:
nlp = Spanish()

# Registra la extensión de atributo del Token, "is_country",
# con el valor por defecto False
Token.set_extension("is_country", default=False)

# Procesa el texto y pon True para el atributo "is_country"
# para el token "España"
doc = nlp("Vivo en España.")
doc[2]._.is_country = True

# Imprime en pantalla el texto del token y el atributo "is_country"
# para todos los tokens
print([(token.text, token._.is_country) for token in doc])

[('Vivo', False), ('en', False), ('España', True), ('.', False)]


In [17]:
# Define el método
def to_html(span, tag):
    # Envuelve el texto del span en un HTML tag y devuélvelo
    return f"<{tag}>{span.text}</{tag}>"


# Registra la extensión de propiedad del Span, "to_html",
# con el método "to_html"
Span.set_extension("to_html", method=to_html)

In [15]:
# Procesa el texto y llama el método "to_html"en el span
# con el nombre de tag "strong"
doc = nlp("Hola mundo, esto es una frase.")
span = doc[0:2]
print(span._.to_html("body"))


<body>Hola mundo</body>


In [18]:
nlp = spacy.load("es_core_news_sm")

def get_wikipedia_url(span):
    # Obtén la URL de Wikipedia si el span tiene uno de los siguientes labels
    if span.label_ in ("PER", "ORG", "LOC"):
        entity_text = span.text.replace(" ", "_")
        return "https://es.wikipedia.org/w/index.php?search=" + entity_text


# Añade la extensión del Span, wikipedia_url, usando el getter get_wikipedia_url
Span.set_extension("wikipedia_url", getter=get_wikipedia_url)

doc = nlp(
    "Antes de finalizar 1976, el interés de David Bowie en la "
    "floreciente escena musical alemana, le llevó a mudarse a "
    "Alemania para revitalizar su carrera."
)
for ent in doc.ents:
    # Imprime en pantalla el texto y la URL de Wikipedia de la entidad
    print(ent.text, ent._.wikipedia_url)

David Bowie https://es.wikipedia.org/w/index.php?search=David_Bowie
Alemania https://es.wikipedia.org/w/index.php?search=Alemania


In [19]:
with open("../resources/notebook/countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())

with open("../resources/notebook/capitals.json", encoding="utf8") as f:
    CAPITALS = json.loads(f.read())

nlp = Spanish()
matcher = PhraseMatcher(nlp.vocab)
matcher.add("COUNTRY", None, *list(nlp.pipe(COUNTRIES)))

@Language.component("countries_component")
def countries_component(doc):
    # Crea un Span de entidades con el label "LOC" para todos los resultados
    matches = matcher(doc)
    doc.ents = [Span(doc, start, end, label="LOC") for match_id, start, end in matches]
    return doc


# Añade el componente al pipeline
nlp.add_pipe("countries_component", name="countries_matcher")
print(nlp.pipe_names)

# El getter que busca el texto del span en un diccionario de ciudades
# capitales de países
get_capital = lambda span: CAPITALS.get(span.text)

# Registra la extensión de atributo del Span, "capital", con el 
# getter get_capital
Span.set_extension("capital", getter=get_capital)

# Procesa el texto e imprime en pantalla el texto de la entidad,
# el label y los atributos "capital"
doc = nlp(
    "La República Checa podría ayudar a la República Eslovaca "
    "a proteger su espacio aéreo"
)
print([(ent.text, ent.label_, ent._.capital) for ent in doc.ents])

['countries_matcher']
[('República Checa', 'LOC', 'Prague'), ('República Eslovaca', 'LOC', 'Bratislava')]


### Procesando streams

In [35]:
nlp = spacy.load("es_core_news_sm")

with open("../resources/notebook/tweets.json", encoding="utf8") as f:
    TEXTS = json.loads(f.read())

for doc in nlp.pipe(TEXTS):
    print([token.text for token in doc if token.pos_ == "VERB"])

['desaprovechar', 'comprar', 'compré', 'lastima', 'amo']
['quiere']
['amo']
['hacerme', 'esperar', 'llegado']
['Tengo']


In [36]:
docs = list(nlp.pipe(TEXTS))
entities = [doc.ents for doc in docs]
print(*entities)

(Ahora, Walmart, McDonalds, McDonalds) (Dante, Meal, McMenu, Quesoburguesa, McDonalds) (McDonalds,) (Mcdonalds,) (Tengo, McDonalds)


In [38]:
people = ["David Bowie", "Angela Merkel", "Lady Gaga"]

# Crea una lista de patrones para el PhraseMatcher
patterns = list(nlp.pipe(people))

[David Bowie, Angela Merkel, Lady Gaga]


In [42]:
with open("../resources/notebook/bookquotes.json", encoding="utf8") as f:
    DATA = json.loads(f.read())

nlp = Spanish()

# Registra la extensión del Doc, "author" (por defecto None)
Doc.set_extension("author", default=None)

# Registra la extensión del Doc, "book" (por defecto None)
Doc.set_extension("book", default=None)

In [45]:
for doc, context in nlp.pipe(DATA, as_tuples=True):
    # Añade los atributos doc._.book y doc._.author desde el contexto
    doc._.book = context["book"]
    doc._.author = context["author"]

    # Imprime en pantalla el texto y los datos del atributo personalizado
    print(f"{doc.text}\n — '{doc._.book}' by {doc._.author}\n")

Muchos años después, frente al pelotón de fusilamiento, el coronel Aureliano Buendía había de recordar aquella tarde remota en que su padre lo llevó a conocer el hielo.
 — 'Cien años de soledad' by Gabriel García Márquez

Tenía la ternura torpe de quien nunca ha sido amado y debe improvisar.
 — 'La casa de los espíritus' by Isabel Allende

Andábamos sin buscarnos, pero sabiendo que andábamos para encontrarnos.
 — 'Rayuela' by Julio Cortázar

Cuando fui leona nunca recordé, cómo pude un día mariposa ser.
 — 'Así' by Alfonsina Storni

Los dioses hicieron de maíz a las madres y a los padres. Con maíz amarillo y maíz blanco amasaron su carne.
 — 'Memoria del fuego' by Eduardo Galeano

Y no quiero llantos. La muerte hay que mirarla cara a cara. ¡Silencio!
 — 'La casa de Bernarda Alba' by Federico García Lorca



In [46]:
nlp = spacy.load("es_core_news_sm")
text = (
    "Chick-fil-A es una cadena de restaurantes de comida rápida "
    "americana con sede en la ciudad de College Park, Georgia, "
    "especializada en sándwiches de pollo."
)

# Únicamente convierte el texto en tokens
doc = nlp.make_doc(text)
print([token.text for token in doc])

['Chick-fil-A', 'es', 'una', 'cadena', 'de', 'restaurantes', 'de', 'comida', 'rápida', 'americana', 'con', 'sede', 'en', 'la', 'ciudad', 'de', 'College', 'Park', ',', 'Georgia', ',', 'especializada', 'en', 'sándwiches', 'de', 'pollo', '.']


In [49]:
nlp = spacy.load("es_core_news_sm")
text = (
    "Chick-fil-A es una cadena de restaurantes de comida rápida "
    "americana con sede en la ciudad de College Park, Georgia, "
    "especializada en sándwiches de pollo."
)

# Deshabilita el tagger y el parser
with nlp.disable_pipes("morphologizer", "parser"):
    # Procesa el texto
    doc = nlp(text)
    # Imprime las entidades del doc en pantalla
    print(doc.ents)

(College Park, Georgia)


## Capítulo 4: Entrenando un modelo de red neuronal

In [4]:
with open("../resources/notebook/adidas.json", encoding="utf8") as f:
    TEXTS = json.loads(f.read())

nlp = Spanish()
matcher = Matcher(nlp.vocab)

# Dos tokens que en minúsculas encuentran "adidas" y "zx"
pattern1 = [{"LOWER": "adidas"}, {"LOWER": "zx"}]

# Token que en minúsculas encuentra "adidas" y un dígito
pattern2 = [{"LOWER": "adidas"}, {"IS_DIGIT": True}]

# Añade los patrones al matcher y revisa el resultado
matcher.add("ROPA", [pattern1, pattern2])
for doc in nlp.pipe(TEXTS):
    print([doc[start:end] for match_id, start, end in matcher(doc)])

[adidas ZX]
[adidas ZX]
[adidas ZX]
[adidas 8000, adidas 4000]
[]


In [5]:
TRAINING_DATA = []

# Crea un objeto Doc para cada texto en TEXTS
for doc in nlp.pipe(TEXTS):
    # Encuentra en el doc y crea una lista de los spans resultantes
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    # Obtén los tuples (carácter de inicio, carácter del final, label) resultantes
    entities = [(span.start_char, span.end_char, "ROPA") for span in spans]
    # Da formato a los resultados como tuples con (doc.text, entidades)
    training_example = (doc.text, {"entities": entities})
    # Añade el ejemplo a los datos de entrenamiento
    TRAINING_DATA.append(training_example)

print(*TRAINING_DATA, sep="\n")

('Cómo pre-ordenar los adidas ZX', {'entities': [(21, 30, 'ROPA')]})
('Los nuevos adidas ZX vienen en camino', {'entities': [(11, 20, 'ROPA')]})
('Debería pagar €200 por un par de adidas ZX?', {'entities': [(33, 42, 'ROPA')]})
('Cuál es la diferencia entre los adidas 8000 y los adidas 4000?', {'entities': [(32, 43, 'ROPA'), (50, 61, 'ROPA')]})
('Necesito nuevas zapatillas! ¿Qué me recomiendan?', {'entities': []})


### Creación del modelo

In [9]:
#Crea un modelo "es" en blanco
nlp = spacy.blank("es")

# Crea un nuevo entity recognizer y añádelo al pipeline
ner = nlp.add_pipe("ner")

# Añade el label "ROPA" al entity recognizer
ner.add_label("ROPA")

1

In [19]:
with open("../resources/notebook/ropa.json", encoding="utf8") as f:
    TRAINING_DATA = json.loads(f.read())

# Comienza el entrenamiento
nlp.begin_training()
optimizer = nlp.initialize()

# Haz un loop por 10 iteraciones
for itn in range(100):
    # Mezcla los datos de entrenamiento
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Crea lotes con los ejemplos e itera sobre ellos
    for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
        for text, annotations in batch:
            # create Example
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            # Actualiza el modelo
            nlp.update([example], losses=losses, sgd=optimizer)

    if itn%10 == 0: print(losses)

{'ner': 28.144546270370483}
{'ner': 1.009810970427832e-08}
{'ner': 1.1983537725710967e-09}
{'ner': 5.776937901592609e-10}
{'ner': 2.2641713326645672e-10}
{'ner': 1.9263452288965482e-10}
{'ner': 1.57823824010173e-10}
{'ner': 9.846804359455153e-11}
{'ner': 1.0698315661136817e-10}
{'ner': 9.534702815254137e-11}
